In [1]:
import torch as t
from torch import nn,optim
from torch.autograd import Variable
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import time

import visdom

vis = visdom.Visdom(env = u'Front_side_face')



In [3]:
class gaussion_kernel(nn.Module):
    def __init__(self,inchannel=1,outchannel=1,stride=1):
        super().__init__()
        self.gaus_conv1 = nn.Conv2d(inchannel,outchannel,(3,3),stride,padding=0,dilation=1,bias=False)    #这个padding可以保证前后图片大小相等。
        self.gaus_conv1.weight.data = (t.Tensor([[1/16,1/8,1/16],[1/8,1/4,1/8],[1/16,1/8,1/16]])).view(1,1,3,3)     #高斯模板
        
        
    def forward(self,x):
        #print(x[:,0,:,:].shape)   #dim=3            #这个维数不够，所以下面要加上unsqueeze(1)
        x1 = self.gaus_conv1(x[:,0,:,:].unsqueeze(1))    #参考P144
        x2 = self.gaus_conv1(x[:,1,:,:].unsqueeze(1))
        x3 = self.gaus_conv1(x[:,2,:,:].unsqueeze(1))
        x = t.cat((x1,x2,x3),1)
        #print(x.shape)
        
        return x

In [4]:
class gaussion_kernel0(nn.Module):
    def __init__(self,inchannel=1,outchannel=1,stride=1):
        super().__init__()
        self.gaus_conv1 = nn.Conv2d(inchannel,outchannel,(3,3),stride,padding=0,dilation=1,bias=False)    #这个padding可以保证前后图片大小相等。
        self.gaus_conv1.weight.data = (t.Tensor([[1/16,1/8,1/16],[1/8,1/4,1/8],[1/16,1/8,1/16]])).view(1,1,3,3)     #高斯模板
        
        
    def forward(self,x):
        #print(x[:,0,:,:].shape)   #dim=3            #这个维数不够，所以下面要加上unsqueeze(1)
        x1 = self.gaus_conv1(x[:,0,:,:].unsqueeze(1))    #参考P144
        x2 = self.gaus_conv1(x[:,1,:,:].unsqueeze(1))
        x3 = self.gaus_conv1(x[:,2,:,:].unsqueeze(1))
        x4 = self.gaus_conv1(x[:,3,:,:].unsqueeze(1))    
        x5 = self.gaus_conv1(x[:,4,:,:].unsqueeze(1))
        x6 = self.gaus_conv1(x[:,5,:,:].unsqueeze(1))
        x = t.cat((x1,x2,x3,x4,x5,x6),1)
        #print(x.shape)
        
        return x

In [5]:
class gaussion_kernel1(nn.Module):
    def __init__(self,inchannel=1,outchannel=1,stride=1):
        super().__init__()
        self.gaus_conv1 = nn.Conv2d(inchannel,outchannel,(3,3),stride,padding=0,dilation=1,bias=False)    #这个padding可以保证前后图片大小相等。
        self.gaus_conv1.weight.data = (t.Tensor([[1/16,1/8,1/16],[1/8,1/4,1/8],[1/16,1/8,1/16]])).view(1,1,3,3)     #高斯模板
        
        
    def forward(self,x):
        #print(x[:,0,:,:].shape)   #dim=3            #这个维数不够，所以下面要加上unsqueeze(1)
        x1 = self.gaus_conv1(x[:,0,:,:].unsqueeze(1))    #参考P144
        x2 = self.gaus_conv1(x[:,1,:,:].unsqueeze(1))
        x3 = self.gaus_conv1(x[:,2,:,:].unsqueeze(1))
        x4 = self.gaus_conv1(x[:,3,:,:].unsqueeze(1))    
        x5 = self.gaus_conv1(x[:,4,:,:].unsqueeze(1))
        x6 = self.gaus_conv1(x[:,5,:,:].unsqueeze(1))
        x7 = self.gaus_conv1(x[:,6,:,:].unsqueeze(1))
        x8 = self.gaus_conv1(x[:,7,:,:].unsqueeze(1))
        
        x9 = self.gaus_conv1(x[:,0,:,:].unsqueeze(1))    #参考P144
        x10 = self.gaus_conv1(x[:,1,:,:].unsqueeze(1))
        x11 = self.gaus_conv1(x[:,2,:,:].unsqueeze(1))
        x12 = self.gaus_conv1(x[:,3,:,:].unsqueeze(1))    
        #x13 = self.gaus_conv1(x[:,4,:,:].unsqueeze(1))
        #x14 = self.gaus_conv1(x[:,5,:,:].unsqueeze(1))
        #x15 = self.gaus_conv1(x[:,6,:,:].unsqueeze(1))
        #x16 = self.gaus_conv1(x[:,7,:,:].unsqueeze(1))
        x = t.cat((x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12),1)
        #print(x.shape)
        
        return x

In [4]:
#这个cell没用，不用执行
gaus_layer = gaussion_kernel()
gaus_layer1 = gaussion_kernel1()
list(gaus_layer.parameters())     #说明我的方法还是可以的，存在可学习的参数。

[Parameter containing:
 (0 ,0 ,.,.) = 
   0.0625  0.1250  0.0625
   0.1250  0.2500  0.1250
   0.0625  0.1250  0.0625
 [torch.FloatTensor of size 1x1x3x3]]

In [11]:
#有3高斯层，6,12,24
layers1 = 12
layers2 = 24
class CNN_Net(nn.Module):
    def __init__(self):
        super(CNN_Net,self).__init__()
        self.layer0 = nn.Sequential(   #这一层只是进行尺寸变化。
                                    #gaussion_kernel(),
                                    nn.Conv2d(3,3,(5,5),stride=2,padding=0),     #这样可以保证输入120*120的图片时，输出为60*60
                                    #nn.ReLU(True)
                                    )
        self.layer1 = nn.Sequential(
                                    #gaussion_kernel0(),
                                    nn.Conv2d(3,layers1,(7,7),stride=1,padding=0),    #为了减少0填充对于结果的影响，不进行边缘填充，反正层数少。
                                    nn.BatchNorm2d(layers1),
                                    nn.ReLU(True),
                                    nn.AvgPool2d(2,2)
                                    #nn.AdaptiveAvgPool2d(27)
                                    )
        
        self.layer2 = nn.Sequential(
                                    #gaus_layer(),
                                    #gaussion_kernel1(),
                                    nn.Conv2d(layers1,layers2,(5,5),stride=1,padding=0),
                                    nn.BatchNorm2d(layers2),
                                    nn.ReLU(True),
                                    nn.AvgPool2d(3,3)
                                    )
        
        
        #self.gaus_layer2 = gaus_layer()
        #self.conv3 = nn.Conv2d(12,24,(3,3),stride=1,padding=1)
        #self.pool2 = nn.AvgPool2d(2,2)
        self.full_layer1 = nn.Sequential(                   #因为输入是16*6*6
                                    nn.Linear(1176,64),     
                                    nn.BatchNorm1d(64),
                                    nn.ReLU(True)
                                    )
        self.full_layer2 = nn.Sequential(                  
                                    nn.Linear(64,32),     
                                    nn.BatchNorm1d(32),
                                    nn.ReLU(True)
                                    )
        self.full_layer3 = nn.Sequential(                  
                                    nn.Linear(32,16),     
                                    nn.BatchNorm1d(16),
                                    nn.ReLU(True)
                                    )
        self.full_layer_output = nn.Sequential(                   
                                    nn.Linear(16,2),     
                                    
                                    )
        #self.softmax = nn.Softmax()
        
    def forward(self,x):
        x = self.layer0(x)
        x = self.layer1(x)
        x = self.layer2(x)
        #print("x.shape",x.shape)
        x = x.view(x.size()[0], -1)
        #print(type(x))
        #print("x.shape1",x.shape)
        x = self.full_layer1(x)
        x = self.full_layer2(x)
        x = self.full_layer3(x)
        x = self.full_layer_output(x)
        #x = self.softmax(x)
        
        return x
        

In [13]:
#有高斯处理，3，12,24
class CNN_Net1(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer0 = nn.Sequential(   #这一层只是进行尺寸变化。
                                    #gaus_layer(),
                                    gaussion_kernel(),
                                    nn.Conv2d(3,3,(5,5),stride=2,padding=0)     #这样可以保证输入120*120的图片时，输出为60*60
                                    )
        self.layer1 = nn.Sequential(
                                    #gaus_layer(),
                                    gaussion_kernel(),
                                    nn.Conv2d(3,8,(7,7),stride=1,padding=0),    #为了减少0填充对于结果的影响，不进行边缘填充，反正层数少。
                                    nn.BatchNorm2d(8),
                                    nn.ReLU(True),
                                    nn.AvgPool2d(2,2)
                                    )
        
        self.layer2 = nn.Sequential(
                                    #gaus_layer(),
                                    gaussion_kernel1(),
                                    nn.Conv2d(6,12,(5,5),stride=1,padding=0),
                                    nn.BatchNorm2d(12),
                                    nn.ReLU(True),
                                    nn.AvgPool2d(3,3)
                                    )
        
        
        #self.gaus_layer2 = gaus_layer()
        #self.conv3 = nn.Conv2d(12,24,(3,3),stride=1,padding=1)
        #self.pool2 = nn.AvgPool2d(2,2)
        self.full_layer1 = nn.Sequential(                   #因为输入是12*6*6    去掉高斯层为12*7*7
                                    nn.Linear(432,64),     
                                    nn.BatchNorm1d(64),
                                    nn.ReLU(True)
                                    )
        self.full_layer2 = nn.Sequential(                  
                                    nn.Linear(64,32),     
                                    nn.BatchNorm1d(32),
                                    nn.ReLU(True)
                                    )
        self.full_layer_output = nn.Sequential(                   
                                    nn.Linear(32,2),     
                                    
                                    )
        self.softmax = nn.Softmax()
        
    def forward(self,x):
        x = self.layer0(x)
        x = self.layer1(x)
        x = self.layer2(x)
        #print("x.shape",x.shape)
        x = x.view(x.size()[0], -1)
        #print(type(x))
        #print("x.shape1",x.shape)
        x = self.full_layer1(x)
        x = self.full_layer2(x)
        x = self.full_layer_output(x)
        x = self.softmax(x)
        
        return x
        

In [12]:


normalize = transforms.Normalize(mean=[0.5,0.5,0.5],std=[0.5,0.5,0.5])
transform = transforms.Compose([
                                transforms.RandomRotation(20),         #图像旋转的角度
                                transforms.RandomHorizontalFlip(),    #水平方向翻转
                                transforms.ToTensor(),
                                normalize,
                                ])
dataset = ImageFolder("D:\\newfolder51",transform=transform)
#dataset.imgs   

#print(dataset.shape)
#print(dataset.mean(axis=(0,1,2))/255)
#print(dataset.std(axis=(0,1,2))/255)
dataset[9][0]


( 0 ,.,.) = 
 -1.0000 -1.0000 -1.0000  ...  -1.0000 -1.0000 -1.0000
 -1.0000 -1.0000 -1.0000  ...  -1.0000 -1.0000 -1.0000
 -1.0000 -1.0000 -1.0000  ...  -1.0000 -1.0000 -1.0000
           ...             ⋱             ...          
 -1.0000 -1.0000 -1.0000  ...  -1.0000 -1.0000 -1.0000
 -1.0000 -1.0000 -1.0000  ...  -1.0000 -1.0000 -1.0000
 -1.0000 -1.0000 -1.0000  ...  -1.0000 -1.0000 -1.0000

( 1 ,.,.) = 
 -1.0000 -1.0000 -1.0000  ...  -1.0000 -1.0000 -1.0000
 -1.0000 -1.0000 -1.0000  ...  -1.0000 -1.0000 -1.0000
 -1.0000 -1.0000 -1.0000  ...  -1.0000 -1.0000 -1.0000
           ...             ⋱             ...          
 -1.0000 -1.0000 -1.0000  ...  -1.0000 -1.0000 -1.0000
 -1.0000 -1.0000 -1.0000  ...  -1.0000 -1.0000 -1.0000
 -1.0000 -1.0000 -1.0000  ...  -1.0000 -1.0000 -1.0000

( 2 ,.,.) = 
 -1.0000 -1.0000 -1.0000  ...  -1.0000 -1.0000 -1.0000
 -1.0000 -1.0000 -1.0000  ...  -1.0000 -1.0000 -1.0000
 -1.0000 -1.0000 -1.0000  ...  -1.0000 -1.0000 -1.0000
           ...         

In [13]:
batch_size = 64
learning_rate = 1e-4
num_epoches = 500
weight_decay = 1e-4

In [14]:
dataloader = DataLoader(dataset,batch_size=batch_size,shuffle=True)
#ataiter = iter(dataloader)
#imgs,labels = next(dataiter)
#imgs.size()

In [15]:
model = CNN_Net()

In [16]:
if t.cuda.is_available():
    model = model.cuda()
    
criterion = nn.CrossEntropyLoss()
criterion.cuda() 
#optimizer = optim.Adam(model.parameters(),lr=learning_rate,weight_decay=weight_decay)  

CrossEntropyLoss(
)

In [11]:
optimizer = optim.Adam([
                        {'params':model.layer0[0].parameters(),'lr':0.0},
                        {'params':model.layer0[1].parameters(),'lr':learning_rate},
                       
                        {'params':model.layer1[0].parameters(),'lr':0},
                        {'params':model.layer1[1].parameters(),'lr':learning_rate},
                        {'params':model.layer1[2].parameters(),'lr':learning_rate},
                        {'params':model.layer1[3].parameters(),'lr':learning_rate},
                        {'params':model.layer1[4].parameters(),'lr':learning_rate},
    
                        {'params':model.layer2[0].parameters(),'lr':0},
                        {'params':model.layer2[1].parameters(),'lr':learning_rate},
                        {'params':model.layer2[2].parameters(),'lr':learning_rate},
                        {'params':model.layer2[3].parameters(),'lr':learning_rate},
                        {'params':model.layer2[4].parameters(),'lr':learning_rate},
    
                        {'params':model.full_layer1[0].parameters(),'lr':learning_rate},
                        {'params':model.full_layer1[1].parameters(),'lr':learning_rate},
                        {'params':model.full_layer1[2].parameters(),'lr':learning_rate},
    
                        {'params':model.full_layer2[0].parameters(),'lr':learning_rate},
                        {'params':model.full_layer2[1].parameters(),'lr':learning_rate},
                        {'params':model.full_layer2[2].parameters(),'lr':learning_rate},
    
                        {'params':model.full_layer_output[0].parameters(),'lr':learning_rate},
    
                        #{'params':model.softmax[0].parameters(),'lr':learning_rate}
                       ],weight_decay=weight_decay)

In [17]:
#去掉高斯层时的优化函数
optimizer = optim.Adam([
                        #{'params':model.layer0[0].parameters(),'lr':0.0},
                        {'params':model.layer0[0].parameters(),'lr':learning_rate},
                       
                        #{'params':model.layer1[0].parameters(),'lr':0},
                        {'params':model.layer1[0].parameters(),'lr':learning_rate},
                        {'params':model.layer1[1].parameters(),'lr':learning_rate},
                        {'params':model.layer1[2].parameters(),'lr':learning_rate},
                        {'params':model.layer1[3].parameters(),'lr':learning_rate},
                        #{'params':model.layer1[4].parameters(),'lr':learning_rate},
    
                        #{'params':model.layer2[0].parameters(),'lr':0},
                        {'params':model.layer2[0].parameters(),'lr':learning_rate},
                        {'params':model.layer2[1].parameters(),'lr':learning_rate},
                        {'params':model.layer2[2].parameters(),'lr':learning_rate},
                        {'params':model.layer2[3].parameters(),'lr':learning_rate},
                        #{'params':model.layer2[4].parameters(),'lr':learning_rate},
    
                        {'params':model.full_layer1[0].parameters(),'lr':learning_rate},
                        {'params':model.full_layer1[1].parameters(),'lr':learning_rate},
                        {'params':model.full_layer1[2].parameters(),'lr':learning_rate},
    
                        {'params':model.full_layer2[0].parameters(),'lr':learning_rate},
                        {'params':model.full_layer2[1].parameters(),'lr':learning_rate},
                        {'params':model.full_layer2[2].parameters(),'lr':learning_rate},
    
                        {'params':model.full_layer_output[0].parameters(),'lr':learning_rate},
    
                        #{'params':model.softmax[0].parameters(),'lr':learning_rate}
                       ],weight_decay=weight_decay)

In [18]:
#train
print("training start time  <===>  ",time.localtime())       #
sum_loss = t.Tensor([0])     #多一个0元素，不会影响平均值
for epoch in range(0,num_epoches):
    for i,data in enumerate(dataloader,0):   #表示i 从0开始
        data_batch, label = data
        data_batch, label = Variable(data_batch.float()).cuda(),Variable(label).cuda()
        output = model(data_batch)
        #print("output.shape",output.shape)
        #print(output.cpu())
        optimizer.zero_grad()
        loss = criterion(output,label)
        loss.backward()            #我终于知道，我的图为什么不下降了，我忘记反向传播了。。。。。。。。。。。。
        optimizer.step()
        #sum_loss.append(loss)
        sum_loss = t.cat((sum_loss,loss.cpu().data),0)
        x = i+epoch*len(dataloader)      #此时的x还只是个常数，不是列表，也不是np.array()
        
        if x%50 == 49:
            loss_aver = sum_loss.mean()
            sum_loss = t.Tensor([0])
            #vis.line(X=x, Y=loss.cpu().data, win='face_angle',opts={'title': 'loss-batch', 'xlabel':'x','ylabel':'y'},update='append' if i>=0 else None)
            vis.line(X=t.Tensor([x]), Y=t.Tensor([loss_aver]), win='Front_side_face',opts={'title': 'loss-batch', 'xlabel':'batch_num','ylabel':'loss_aver'},update='append' if x>49 else None)
            #vis.updateTrace(X=t.Tensor([x]),Y=t.Tensor([loss_aver]),win='blur_degrees', name='1e-4_3.12.24_NoGaussion_NoReLU')
print("training end time  <===>  ",time.localtime()) 
t.save(model.state_dict(),"C:\\Users\\SiChen\\face_sys\\front_side_face\\model_save\\Front_side_CNN_network_500_1e-4_3.12.24_4full-layers_NoGaussion_NoSoftmax.pth")

training start time  <===>   time.struct_time(tm_year=2018, tm_mon=5, tm_mday=30, tm_hour=15, tm_min=53, tm_sec=16, tm_wday=2, tm_yday=150, tm_isdst=0)
training end time  <===>   time.struct_time(tm_year=2018, tm_mon=5, tm_mday=31, tm_hour=13, tm_min=30, tm_sec=28, tm_wday=3, tm_yday=151, tm_isdst=0)


In [20]:
#实验time.localtime()的使用方法
print(time.localtime())
time.sleep(3)
print("training end time  <===>  ",time.localtime()) 
time.sleep(3)
time.localtime()

time.struct_time(tm_year=2018, tm_mon=4, tm_mday=30, tm_hour=12, tm_min=50, tm_sec=22, tm_wday=0, tm_yday=120, tm_isdst=0)
training end time  <===>   time.struct_time(tm_year=2018, tm_mon=4, tm_mday=30, tm_hour=12, tm_min=50, tm_sec=25, tm_wday=0, tm_yday=120, tm_isdst=0)


time.struct_time(tm_year=2018, tm_mon=4, tm_mday=30, tm_hour=12, tm_min=50, tm_sec=28, tm_wday=0, tm_yday=120, tm_isdst=0)

# 保存和加载模型

In [10]:
t.save(model.state_dict(),"C:\\Users\\SiChen\\face_sys\\front_side_face\\model_save\\Front_side_CNN_network_500_1e-4_3.12.24_NoGaussion_softmax.pth")

In [18]:
#加载模型必须在，前面定义模型之后，即在前面的model = ....之后才能执行这一行。
model.load_state_dict(t.load("C:\\Users\\SiChen\\face_sys\\FC_network\\model_save\\cnn_blur_network_500_1e-4_softmax.pth"))

# 以下是测试部分

In [11]:
normalize = transforms.Normalize(mean=[0.5,0.5,0.5],std=[0.5,0.5,0.5])
transform = transforms.Compose([
                                #transforms.RandomRotation(20),         #图像旋转的角度
                                #transforms.RandomHorizontalFlip(),    #水平方向翻转
                                transforms.ToTensor(),
                                normalize,
                                ])
test_dataset = ImageFolder("D:\\newfolder52",transform=transform)
test_dataloader = DataLoader(test_dataset,batch_size=batch_size,shuffle=True)

In [12]:
model.cuda()
model.eval()
test_right_num = 0
test_right_num = t.Tensor([test_right_num]).int()
test_right_num = Variable(test_right_num)
for data in test_dataloader:
    data_batch,label = data
    data_batch,label = Variable(data_batch.float(),volatile=True).cuda(),Variable(label,volatile=True).cuda()
    out = model(data_batch)
    _,pred = t.max(out.cpu(),1)
    num_correct = (pred==label.cpu()).sum()
    #print(num_correct)
    num_correct = num_correct.int()     #是torch.ByteTensor转为torch.IntTensor类型。size为1
    test_right_num = test_right_num + num_correct
    #print("test_right_num",test_right_num)
    
acc = (float(test_right_num))/len(test_dataset)
print(test_right_num)
print(len(test_dataset))
print(acc)

d:\Anaconda3\envs\pytorch\lib\site-packages\ipykernel_launcher.py:61: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Variable containing:
 2217
[torch.IntTensor of size 1]

2230
0.9941704035874439


In [13]:
from PIL import Image
import shutil

front_wrong_face = "D:\\newfolder53\\front_wrong_face"          #保存错误的人脸的文件夹
side_wrong_face = "D:\\newfolder53\\side_wrong_face"

model.cuda()
model.eval()
test_right_num = 0
test_right_num = t.Tensor([test_right_num]).int()
test_right_num = Variable(test_right_num)

normalize = transforms.Normalize(mean=[0.5,0.5,0.5],std=[0.5,0.5,0.5])
transform = transforms.Compose([
                                #transforms.RandomRotation(20),         #图像旋转的角度
                                #transforms.RandomHorizontalFlip(),    #水平方向翻转
                                transforms.ToTensor(),
                                normalize,
                                ])
test_dataset = ImageFolder("D:\\newfolder52")

for img0, label0 in test_dataset.imgs:            #!!!!!!!!!!!!!!!!!!!!!只有这样我才能得到需要的名字。
    #print(img0)
    #print(label0)
    img = Image.open(img0)                  #书本P108
    img = transform(img)         
    img = img.unsqueeze(0)                  #书本P109
    label = Variable(t.Tensor([label0]))
    label = label.long()
    
    img, label = Variable(img.float()).cuda(),label.cuda()
    out = model(img)
    #print(out.data)
    _,pred = t.max(out.cpu(),1)                 #
    num_correct = (pred==label.cpu()).sum()
    #print(num_correct)
    num_correct = num_correct.int()     #是torch.ByteTensor转为torch.IntTensor类型。size为1
    test_right_num = test_right_num + num_correct
    
    if (pred==label.cpu()).data[0] == 0:   #判断model处理后的结果是否正确。
        if not label0:            #因为clear_face对应的id是0.见前面的test_dataset.class_to_idx
            shutil.copy(img0,front_wrong_face)
            #print("F")
        else:
            #print("F")
            shutil.copy(img0,side_wrong_face)
    

acc = (float(test_right_num))/len(test_dataset)
print(test_right_num)
print(len(test_dataset))
print(acc)

D:\newfolder52\clear_front_face\picture_100_.png
D:\newfolder52\clear_front_face\picture_1013_.png
D:\newfolder52\clear_front_face\picture_1014_.png
D:\newfolder52\clear_front_face\picture_1015_.png
D:\newfolder52\clear_front_face\picture_1016_.png
D:\newfolder52\clear_front_face\picture_101_.png
D:\newfolder52\clear_front_face\picture_1031_.png
D:\newfolder52\clear_front_face\picture_1034_.png
D:\newfolder52\clear_front_face\picture_1039_.png
D:\newfolder52\clear_front_face\picture_1042_.png
D:\newfolder52\clear_front_face\picture_1049_.png
D:\newfolder52\clear_front_face\picture_1050_.png
D:\newfolder52\clear_front_face\picture_1058_.png
D:\newfolder52\clear_front_face\picture_1059_.png
D:\newfolder52\clear_front_face\picture_105_.png
D:\newfolder52\clear_front_face\picture_1060_.png
D:\newfolder52\clear_front_face\picture_1061_.png
D:\newfolder52\clear_front_face\picture_1062_.png
D:\newfolder52\clear_front_face\picture_1063_.png
D:\newfolder52\clear_front_face\picture_1064_.png
D:\

d:\Anaconda3\envs\pytorch\lib\site-packages\ipykernel_launcher.py:61: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


D:\newfolder52\clear_front_face\picture_10_.png
D:\newfolder52\clear_front_face\picture_1101_.png
D:\newfolder52\clear_front_face\picture_1104_.png
D:\newfolder52\clear_front_face\picture_1106_.png
D:\newfolder52\clear_front_face\picture_110_.png
D:\newfolder52\clear_front_face\picture_1111_.png
D:\newfolder52\clear_front_face\picture_1113_.png
D:\newfolder52\clear_front_face\picture_1115_.png
D:\newfolder52\clear_front_face\picture_1116_.png
D:\newfolder52\clear_front_face\picture_1117_.png
D:\newfolder52\clear_front_face\picture_1118_.png
D:\newfolder52\clear_front_face\picture_111_.png
D:\newfolder52\clear_front_face\picture_1120_.png
D:\newfolder52\clear_front_face\picture_1123_.png
D:\newfolder52\clear_front_face\picture_1125_.png
D:\newfolder52\clear_front_face\picture_1126_.png
D:\newfolder52\clear_front_face\picture_1127_.png
D:\newfolder52\clear_front_face\picture_1128_.png
D:\newfolder52\clear_front_face\picture_112_.png
D:\newfolder52\clear_front_face\picture_1131_.png
D:\ne

D:\newfolder52\clear_front_face\picture_142_.png
D:\newfolder52\clear_front_face\picture_1431_.png
D:\newfolder52\clear_front_face\picture_1436_.png
D:\newfolder52\clear_front_face\picture_143_.png
D:\newfolder52\clear_front_face\picture_1441_.png
D:\newfolder52\clear_front_face\picture_1444_.png
D:\newfolder52\clear_front_face\picture_1447_.png
D:\newfolder52\clear_front_face\picture_144_.png
D:\newfolder52\clear_front_face\picture_1450_.png
D:\newfolder52\clear_front_face\picture_1453_.png
D:\newfolder52\clear_front_face\picture_1454_.png
D:\newfolder52\clear_front_face\picture_1457_.png
D:\newfolder52\clear_front_face\picture_145_.png
D:\newfolder52\clear_front_face\picture_1461_.png
D:\newfolder52\clear_front_face\picture_1465_.png
D:\newfolder52\clear_front_face\picture_1469_.png
D:\newfolder52\clear_front_face\picture_146_.png
D:\newfolder52\clear_front_face\picture_1473_.png
D:\newfolder52\clear_front_face\picture_1477_.png
D:\newfolder52\clear_front_face\picture_147_.png
D:\new

D:\newfolder52\clear_front_face\picture_1854_.png
D:\newfolder52\clear_front_face\picture_1855_.png
D:\newfolder52\clear_front_face\picture_1856_.png
D:\newfolder52\clear_front_face\picture_1857_.png
D:\newfolder52\clear_front_face\picture_1858_.png
D:\newfolder52\clear_front_face\picture_1859_.png
D:\newfolder52\clear_front_face\picture_185_.png
D:\newfolder52\clear_front_face\picture_1860_.png
D:\newfolder52\clear_front_face\picture_1861_.png
D:\newfolder52\clear_front_face\picture_1863_.png
D:\newfolder52\clear_front_face\picture_1865_.png
D:\newfolder52\clear_front_face\picture_1866_.png
D:\newfolder52\clear_front_face\picture_1867_.png
D:\newfolder52\clear_front_face\picture_1868_.png
D:\newfolder52\clear_front_face\picture_1869_.png
D:\newfolder52\clear_front_face\picture_186_.png
D:\newfolder52\clear_front_face\picture_1870_.png
D:\newfolder52\clear_front_face\picture_1871_.png
D:\newfolder52\clear_front_face\picture_1873_.png
D:\newfolder52\clear_front_face\picture_1875_.png
D:

...

D:\newfolder52\clear_side_face\picture_2714_.png
D:\newfolder52\clear_side_face\picture_2717_.png
D:\newfolder52\clear_side_face\picture_2722_.png
D:\newfolder52\clear_side_face\picture_2724_.png
D:\newfolder52\clear_side_face\picture_2726_.png
D:\newfolder52\clear_side_face\picture_2728_.png
D:\newfolder52\clear_side_face\picture_2730_.png
D:\newfolder52\clear_side_face\picture_2731_.png
D:\newfolder52\clear_side_face\picture_2733_.png
D:\newfolder52\clear_side_face\picture_2736_.png
D:\newfolder52\clear_side_face\picture_2738_.png
D:\newfolder52\clear_side_face\picture_2740_.png
D:\newfolder52\clear_side_face\picture_2742_.png
D:\newfolder52\clear_side_face\picture_2745_.png
D:\newfolder52\clear_side_face\picture_2746_.png
D:\newfolder52\clear_side_face\picture_2751_.png
D:\newfolder52\clear_side_face\picture_2756_.png
D:\newfolder52\clear_side_face\picture_2758_.png
D:\newfolder52\clear_side_face\picture_2760_.png
D:\newfolder52\clear_side_face\picture_2762_.png
D:\newfolder52\clear

D:\newfolder52\clear_side_face\picture_3895_.png
D:\newfolder52\clear_side_face\picture_3899_.png
D:\newfolder52\clear_side_face\picture_3900_.png
D:\newfolder52\clear_side_face\picture_3903_.png
D:\newfolder52\clear_side_face\picture_3905_.png
D:\newfolder52\clear_side_face\picture_3907_.png
D:\newfolder52\clear_side_face\picture_3959_.png
D:\newfolder52\clear_side_face\picture_4065_.png
D:\newfolder52\clear_side_face\picture_4072_.png
D:\newfolder52\clear_side_face\picture_4075_.png
D:\newfolder52\clear_side_face\picture_4094_.png
D:\newfolder52\clear_side_face\picture_4098_.png
D:\newfolder52\clear_side_face\picture_4101_.png
D:\newfolder52\clear_side_face\picture_4103_.png
D:\newfolder52\clear_side_face\picture_4108_.png
D:\newfolder52\clear_side_face\picture_4111_.png
D:\newfolder52\clear_side_face\picture_4115_.png
D:\newfolder52\clear_side_face\picture_4116_.png
D:\newfolder52\clear_side_face\picture_4131_.png
D:\newfolder52\clear_side_face\picture_4145_.png
D:\newfolder52\clear

D:\newfolder52\clear_side_face\picture_4944_.png
D:\newfolder52\clear_side_face\picture_4945_.png
D:\newfolder52\clear_side_face\picture_4963_.png
D:\newfolder52\clear_side_face\picture_4967_.png
D:\newfolder52\clear_side_face\picture_4971_.png
D:\newfolder52\clear_side_face\picture_4973_.png
D:\newfolder52\clear_side_face\picture_4974_.png
D:\newfolder52\clear_side_face\picture_4977_.png
D:\newfolder52\clear_side_face\picture_4979_.png
D:\newfolder52\clear_side_face\picture_4981_.png
D:\newfolder52\clear_side_face\picture_4987_.png
D:\newfolder52\clear_side_face\picture_4991_.png
D:\newfolder52\clear_side_face\picture_4992_.png
D:\newfolder52\clear_side_face\picture_4996_.png
D:\newfolder52\clear_side_face\picture_5006_.png
D:\newfolder52\clear_side_face\picture_5009_.png
D:\newfolder52\clear_side_face\picture_5012_.png
D:\newfolder52\clear_side_face\picture_5013_.png
D:\newfolder52\clear_side_face\picture_5016_.png
D:\newfolder52\clear_side_face\picture_5017_.png
D:\newfolder52\clear

D:\newfolder52\clear_side_face\picture_5761_.png
D:\newfolder52\clear_side_face\picture_5766_.png
D:\newfolder52\clear_side_face\picture_5767_.png
D:\newfolder52\clear_side_face\picture_5769_.png
D:\newfolder52\clear_side_face\picture_5770_.png
D:\newfolder52\clear_side_face\picture_5773_.png
D:\newfolder52\clear_side_face\picture_5774_.png
D:\newfolder52\clear_side_face\picture_5777_.png
D:\newfolder52\clear_side_face\picture_5778_.png
D:\newfolder52\clear_side_face\picture_5783_.png
D:\newfolder52\clear_side_face\picture_5784_.png
D:\newfolder52\clear_side_face\picture_5785_.png
D:\newfolder52\clear_side_face\picture_5790_.png
D:\newfolder52\clear_side_face\picture_5791_.png
D:\newfolder52\clear_side_face\picture_5792_.png
D:\newfolder52\clear_side_face\picture_5793_.png
D:\newfolder52\clear_side_face\picture_5794_.png
D:\newfolder52\clear_side_face\picture_5801_.png
D:\newfolder52\clear_side_face\picture_5802_.png
D:\newfolder52\clear_side_face\picture_5805_.png
D:\newfolder52\clear

D:\newfolder52\clear_side_face\picture_786_.png
D:\newfolder52\clear_side_face\picture_788_.png
D:\newfolder52\clear_side_face\picture_795_.png
D:\newfolder52\clear_side_face\picture_796_.png
D:\newfolder52\clear_side_face\picture_797_.png
D:\newfolder52\clear_side_face\picture_798_.png
D:\newfolder52\clear_side_face\picture_799_.png
D:\newfolder52\clear_side_face\picture_800_.png
D:\newfolder52\clear_side_face\picture_801_.png
D:\newfolder52\clear_side_face\picture_802_.png
D:\newfolder52\clear_side_face\picture_803_.png
D:\newfolder52\clear_side_face\picture_804_.png
D:\newfolder52\clear_side_face\picture_806_.png
D:\newfolder52\clear_side_face\picture_807_.png
D:\newfolder52\clear_side_face\picture_809_.png
D:\newfolder52\clear_side_face\picture_812_.png
D:\newfolder52\clear_side_face\picture_813_.png
D:\newfolder52\clear_side_face\picture_815_.png
D:\newfolder52\clear_side_face\picture_817_.png
D:\newfolder52\clear_side_face\picture_819_.png
D:\newfolder52\clear_side_face\picture_8

In [26]:
#无用的cell
import models
#dir(models.__builtins__)
print(dir(models))
#dir(models.__path__)

['__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__']
